In [3]:
!pip install pytorch-lightning

In [4]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

#os.chdir("../../..")
import copy
from pathlib import Path
import warnings
import pytorch_lightning as pl
#import lightning.pytorch as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

OSError: [WinError 127] Die angegebene Prozedur wurde nicht gefunden

In [18]:
#from pytorch_forecasting.data.examples import get_stallion_data
import pandas as pd
import numpy as np
#data = get_stallion_data()  # load data as pandas dataframe
data = pd.read_csv('stallion.csv')
""" 
# Only needed for startup
# add time index
data["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month
data["time_idx"] -= data["time_idx"].min()
# add additional features
# categories have to be strings
data["month"] = data.date.dt.month.astype(str).astype("category")
data["log_volume"] = np.log(data.volume + 1e-8)
data["avg_volume_by_sku"] = (
    data
    .groupby(["time_idx", "sku"], observed=True)
    .volume.transform("mean")
)
data["avg_volume_by_agency"] = (
    data
    .groupby(["time_idx", "agency"], observed=True)
    .volume.transform("mean")
)
# we want to encode special days as one variable and 
# thus need to first reverse one-hot encoding
special_days = [
    "easter_day", "good_friday", "new_year", "christmas",
    "labor_day", "independence_day", "revolution_day_memorial",
    "regional_games", "fifa_u_17_world_cup", "football_gold_cup",
    "beer_capital", "music_fest"
]
data[special_days] = (
    data[special_days]
    .apply(lambda x: x.map({0: "-", 1: x.name}))
    .astype("category")
)
"""

' \n# Only needed for startup\n# add time index\ndata["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month\ndata["time_idx"] -= data["time_idx"].min()\n# add additional features\n# categories have to be strings\ndata["month"] = data.date.dt.month.astype(str).astype("category")\ndata["log_volume"] = np.log(data.volume + 1e-8)\ndata["avg_volume_by_sku"] = (\n    data\n    .groupby(["time_idx", "sku"], observed=True)\n    .volume.transform("mean")\n)\ndata["avg_volume_by_agency"] = (\n    data\n    .groupby(["time_idx", "agency"], observed=True)\n    .volume.transform("mean")\n)\n# we want to encode special days as one variable and \n# thus need to first reverse one-hot encoding\nspecial_days = [\n    "easter_day", "good_friday", "new_year", "christmas",\n    "labor_day", "independence_day", "revolution_day_memorial",\n    "regional_games", "fifa_u_17_world_cup", "football_gold_cup",\n    "beer_capital", "music_fest"\n]\ndata[special_days] = (\n    data[special_days]\n    .appl

In [ ]:
#data.to_csv("stallion.csv")

In [22]:
data.head()

,Unnamed: 0,agency,sku,volume,date,industry_volume,soda_volume,avg_max_temp,price_regular,price_actual,...,football_gold_cup,beer_capital,music_fest,discount_in_percent,timeseries,time_idx,month,log_volume,avg_volume_by_sku,avg_volume_by_agency
0,0,Agency_22,SKU_01,52.2720,2013-01-01,492612703,718394219,25.845238,1168.903668,1069.166193,...,-,-,-,8.532566,0,0,1,3.956461,2613.377501,103.805460
1,238,Agency_37,SKU_04,0.0000,2013-01-01,492612703,718394219,26.505000,1852.273642,1611.466298,...,-,-,-,13.000635,5,0,1,-18.420681,1361.511918,0.549900
2,237,Agency_59,SKU_03,812.9214,2013-01-01,492612703,718394219,22.219737,1270.795012,1197.184260,...,-,-,-,5.792496,9,0,1,6.700634,1225.306376,2041.909586
3,236,Agency_11,SKU_01,316.4400,2013-01-01,492612703,718394219,25.360000,1176.155397,1082.757488,...,-,-,-,7.940950,14,0,1,5.757134,2613.377501,125.690220
4,235,Agency_05,SKU_05,420.9093,2013-01-01,492612703,718394219,24.079012,1327.003396,1207.822992,...,-,-,-,8.981168,22,0,1,6.042417,1179.728165,1638.463500


In [5]:
#!pip install torch==1.10.0 pytorch_lightning==1.5.0 opacus==0.14.0 pytorch-forecasting==0.10.3 --user

In [26]:
total_demand = data.groupby(['date']).sum()
total_demand_SKU1 = data[data['sku']=='SKU_01'].groupby(['date']).sum()

C:\Users\Franz Schramm\AppData\Local\Temp\ipykernel_2588\2662174756.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_demand = data.groupby(['date']).sum()
C:\Users\Franz Schramm\AppData\Local\Temp\ipykernel_2588\2662174756.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_demand_SKU1 = data[data['sku']=='SKU_01'].groupby(['date']).sum()


In [29]:
total_demand_SKU1.head()
print(total_demand_SKU1['volume'])

date
2013-01-01    135895.630074
2013-02-01    151682.860500
2013-03-01    167183.221500
2013-04-01    182822.780250
2013-05-01    191781.605250
2013-06-01    166922.658750
2013-07-01    162372.330000
2013-08-01    166792.500000
2013-09-01    136524.577500
2013-10-01    158416.517337
2013-11-01    149315.334079
2013-12-01    183455.664000
2014-01-01    135958.231423
2014-02-01    165563.999250
2014-03-01    185342.148000
2014-04-01    207929.030250
2014-05-01    163688.798250
2014-06-01    177104.205000
2014-07-01    170066.311500
2014-08-01    179567.172000
2014-09-01    168547.827000
2014-10-01    169295.292000
2014-11-01    154309.320000
2014-12-01    191000.047500
2015-01-01    141813.612000
2015-02-01    154303.596000
2015-03-01    225282.915000
2015-04-01    199092.922500
2015-05-01    193412.014500
2015-06-01    177996.747000
2015-07-01    182649.606000
2015-08-01    178917.123000
2015-09-01    176151.684000
2015-10-01    156728.520000
2015-11-01    155066.400000
2015-12-01    1

In [81]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

# The inclusion of the dataset

values = total_demand_SKU1['volume'].to_numpy()
df = pd.DataFrame(values, columns=['Sales'])


# Preprocessing the data
scaler = MinMaxScaler()
sales_data = scaler.fit_transform(df['Sales'].values.reshape(-1, 1))

# Prepare the dataset for training
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    

# Train the model
epochs = 10
fut_pred = 12
train_window = 20

sample_rate = 20 # TODO define dynamically
noise_multiplier = 1.0
max_grad_norm = 1.0
delta = 1e-5

model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


train_inout_seq = create_inout_sequences(torch.FloatTensor(sales_data), train_window)

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i % 25 == 0:
        print(f'Epoch: {i+1:3} Loss: {single_loss.item():10.8f}')

print(f'Epoch: {i+1:3} Loss: {single_loss.item():10.10f}')

# Make predictions
model.eval()

predictions = []
test_inputs_seq = create_inout_sequences(torch.FloatTensor(sales_data), train_window)

for seq, labels in test_inputs_seq:
    with torch.no_grad():
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))
        predictions.append(model(seq).item())
print(predictions)
print(len(predictions))
# Inverse transform the predictions to get the actual values
#actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:]).reshape(-1, 1))
actual_predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

print('Future sales predictions:', actual_predictions)

C:\Python38\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch:   1 Loss: 0.01344522
Epoch:  10 Loss: 0.0182096716
[0.4221948981285095, 0.4162837266921997, 0.41394150257110596, 0.40216755867004395, 0.42415234446525574, 0.40114644169807434, 0.39620453119277954, 0.4490737318992615, 0.4612337350845337, 0.45865434408187866, 0.44080817699432373, 0.4318125247955322, 0.42452743649482727, 0.419669508934021, 0.40379416942596436, 0.39503392577171326, 0.4164280891418457, 0.3815484941005707, 0.3721908926963806, 0.3919822573661804, 0.4193766713142395, 0.431674599647522, 0.43282851576805115, 0.4383569061756134, 0.43699443340301514, 0.41880232095718384, 0.3995160162448883, 0.39338362216949463, 0.39870625734329224, 0.36695605516433716, 0.3675641417503357, 0.3865888714790344, 0.4097031056880951, 0.4376550316810608, 0.43710196018218994, 0.4247936010360718, 0.4200710952281952, 0.41178372502326965, 0.415591835975647, 0.3764619827270508]
40
Future sales predictions: [[160694.18103337]
 [160033.41310645]
 [159771.59248535]
 [158455.46680923]
 [160912.99007519]
 [

In [79]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from opacus import PrivacyEngine

from torch.utils.data import DataLoader

# The inclusion of the dataset

values = total_demand_SKU1['volume'].to_numpy()
df = pd.DataFrame(values, columns=['Sales'])


# Preprocessing the data
scaler = MinMaxScaler()
sales_data = scaler.fit_transform(df['Sales'].values.reshape(-1, 1))

# Prepare the dataset for training
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    


# Train the model
epochs = 10
fut_pred = 12
train_window = 20
batch_size = 20

sample_rate = batch_size/20 # TODO define dynamically
noise_multiplier = 1.0
max_grad_norm = 1.0
delta = 1e-5
epsilon = 1

model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_loader = DataLoader(
    sales_data,
    batch_size=batch_size,
    pin_memory=True,)

# Attach PrivacyEngine to the optimizer
privacy_engine = PrivacyEngine(module=model,max_grad_norm=max_grad_norm,sample_rate=sample_rate,target_delta=delta,
    target_epsilon=epsilon,
    epochs=epochs,)


train_inout_seq = create_inout_sequences(torch.FloatTensor(sales_data), train_window)

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i % 25 == 0:
        print(f'Epoch: {i+1:3} Loss: {single_loss.item():10.8f}')

print(f'Epoch: {i+1:3} Loss: {single_loss.item():10.10f}')

# Make predictions
model.eval()

predictions = []
test_inputs_seq = create_inout_sequences(torch.FloatTensor(sales_data), train_window)

for seq, labels in test_inputs_seq:
    with torch.no_grad():
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))
        predictions.append(model(seq).item())
print(predictions)
print(len(predictions))
# Inverse transform the predictions to get the actual values
#actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:]).reshape(-1, 1))
actual_predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

print('Future sales predictions:', actual_predictions)

C:\Users\Franz Schramm\AppData\Roaming\Python\Python38\site-packages\opacus\privacy_engine.py:645: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  warnings.warn(
C:\Users\Franz Schramm\AppData\Roaming\Python\Python38\site-packages\opacus\privacy_engine.py:229: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(
C:\Python38\lib\site-packages\torch\nn\modules\module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd 

Epoch:   1 Loss: 0.01594876
Epoch:  10 Loss: 0.0173399001
[0.4295169413089752, 0.42190954089164734, 0.4194563329219818, 0.405042439699173, 0.43459588289260864, 0.4043896496295929, 0.39969363808631897, 0.46844205260276794, 0.48066800832748413, 0.47462159395217896, 0.4500143229961395, 0.4383583962917328, 0.42926886677742004, 0.4236525595188141, 0.4041799306869507, 0.3950488269329071, 0.4245602488517761, 0.3798213303089142, 0.37113770842552185, 0.39917948842048645, 0.4341317117214203, 0.4478030204772949, 0.4472029507160187, 0.4527978301048279, 0.4494481682777405, 0.4249824583530426, 0.4008294343948364, 0.3948219120502472, 0.4032851755619049, 0.36359313130378723, 0.36800023913383484, 0.39447021484375, 0.4237481653690338, 0.4575291872024536, 0.4536202549934387, 0.43587687611579895, 0.42925921082496643, 0.4183315634727478, 0.4234873950481415, 0.3733082711696625]
40
Future sales predictions: [[161512.66030527]
 [160662.2829719 ]
 [160388.05625358]
 [158776.82936795]
 [162080.39915997]
 [15870